***MOUNT THE GOOGLE DRIVE***

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**UNZIP THE DATASET**

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/dataset.zip"
extract_path = "/content/drone_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:

# Verify
!ls /content/drone_dataset


dataset


**Install Dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17516 (delta 7), reused 0 (delta 0), pack-reused 17496 (from 4)
Receiving objects: 100% (17516/17516), 16.60 MiB | 25.34 MiB/s, done.
Resolving deltas: 100% (11995/11995), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta

In [ ]:
dataset_yaml = """
train: /content/drone_dataset/dataset/train/images
val: /content/drone_dataset/dataset/valid/images
test: /content/drone_dataset/dataset/test/images

nc: 1
names: ['drone']
"""

with open('/content/yolov5/drone.yaml', 'w') as f:
    f.write(dataset_yaml)

# Optional: Display the file content
!cat /content/yolov5/drone.yaml




train: /content/drone_dataset/dataset/train/images
val: /content/drone_dataset/dataset/valid/images
test: /content/drone_dataset/dataset/test/images

nc: 1
names: ['drone']


***TRAIN THE MODEL***

In [8]:
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 50 --data drone.yaml --weights yolov5s.pt --cache


/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-07-26 05:51:19.253604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753509079.574247    1282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753509079.659706    1282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for 

**DRONE DETECTION FOR IMAGES**

In [9]:
import cv2
from google.colab.patches import cv2_imshow

# Load image from Google Drive
image_path = "/content/drive/MyDrive/drone_test.jpg"  # ← adjust this path
img = cv2.imread(image_path)

# Save as input.jpg for YOLOv5 detect.py
cv2.imwrite("input.jpg", img)

# Preview the image
cv2_imshow(img)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source input.jpg


In [11]:
import os
from IPython.display import Image, display

output_dir = "/content/yolov5/runs/detect"
latest_exp = sorted(os.listdir(output_dir))[-1]
result_path = os.path.join(output_dir, latest_exp, "input.jpg")

display(Image(filename=result_path))


Output hidden; open in https://colab.research.google.com to view.

***DRONE DETECTION FOR VIDEO***

In [14]:
video_path = "/content/drive/MyDrive/drone_test.mp4"

In [24]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source "$video_path"

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/output_fixed.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/240) /content/output_fixed.mp4: 384x640 (no detections), 436.6ms
video 1/1 (2/240) /content/output_fixed.mp4: 384x640 1 drone, 438.3ms
video 1/1 (3/240) /content/output_fixed.mp4: 384x640 1 drone, 448.9ms
video 1/1 (4/240) /content/output_fixed.mp4: 384x640 1 drone, 405.3ms
video 1/1 (5/240) /content/output_fixed.mp4:

In [20]:
!apt-get install ffmpeg -y
!ffmpeg -i /content/yolov5/runs/detect/exp2/drone_test.mp4 -vcodec libx264 -f mp4 /content/output_fixed.mp4


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [27]:
import base64
from IPython.display import HTML

video_path = "/content/output_fixed.mp4"

mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()

HTML(f"""
<video width=640 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


Output hidden; open in https://colab.research.google.com to view.

**FINAL TESTING OF DRONE DETECTION IN VIDEO**

In [28]:
video_path = "/content/drive/MyDrive/drone_testing.mp4"

In [29]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source "$video_path"

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/drone_testing.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/450) /content/drive/MyDrive/drone_testing.mp4: 640x384 1 drone, 434.7ms
video 1/1 (2/450) /content/drive/MyDrive/drone_testing.mp4: 640x384 1 drone, 437.0ms
video 1/1 (3/450) /content/drive/MyDrive/drone_testing.mp4: 640x384 1 drone, 397.5ms
video 1/1 (4/450) /content/drive/MyDrive/drone_testing.mp4: 64

In [31]:
!apt-get install ffmpeg -y
!ffmpeg -i /content/yolov5/runs/detect/exp7/drone_testing.mp4 -vcodec libx264 -f mp4 /content/output_fixed1.mp4


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [32]:
import base64
from IPython.display import HTML

video_path = "/content/output_fixed1.mp4"

mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()

HTML(f"""
<video width=640 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


Output hidden; open in https://colab.research.google.com to view.